In [1]:
import os
import json
import pandas as pd
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from pathlib import Path

part_path = Path("part-2")
part_path.mkdir(exist_ok=True)

raw_path = Path(f"{part_path}/raw")
raw_path.mkdir(exist_ok=True)

processed_path = Path(f"{part_path}/processed")
processed_path.mkdir(exist_ok=True)

submission_path = Path(f"{part_path}/submission")
submission_path.mkdir(exist_ok=True)

In [3]:
def stringify_array(arr):
    return "'" + "','".join(arr) + "'"

In [4]:
def remove_str(projects):
    projects = [p.replace('https://github.com/', '') for p in projects]
    return projects

In [5]:
def remove_str_df(df):
    df['project_a'] = df['project_a'].str.replace('https://github.com/', '')
    df['project_b'] = df['project_b'].str.replace('https://github.com/', '')
    return df


In [6]:
def get_dataset():
    if str(part_path) == 'part-1':
        # repository_url = (
        #     "https://raw.githubusercontent.com/deepfunding/mini-contest/refs/heads/main/"
        # )
        
        # df_train = pd.read_csv(f"{repository_url}/dataset.csv")
        # df_test = pd.read_csv(f"{repository_url}/test.csv")
        # df_train.to_csv(f"{part_path}/train.csv", index=False)
        # df_test.to_csv(f"{part_path}/test.csv", index=False)
        df_train = pd.read_csv(f"{part_path}/train.csv")
        df_test = pd.read_csv(f"{part_path}/test.csv")
        
    elif str(part_path) == 'part-2':
        df_train = pd.read_csv(f"{part_path}/train.csv")
        df_test = pd.read_csv(f"{part_path}/test.csv")
     
    return df_train, df_test

In [7]:
df_train, df_test = get_dataset()

In [8]:
projects = pd.concat([
    df_train['project_a'],
    df_train['project_b'],
    df_test['project_a'],
    df_test['project_b']
]).unique().tolist()

In [9]:
projects[:5]

['https://github.com/mochajs/mocha',
 'https://github.com/chzyer/readline',
 'https://github.com/gulpjs/gulp',
 'https://github.com/webpack/webpack',
 'https://github.com/redux-saga/redux-saga']

In [10]:
len(projects)

117

In [11]:
trimed_projects = remove_str(projects)

In [12]:
df_train.quarter.unique()

array(['2016-04', '2016-07', '2016-10', '2017-01', '2017-04', '2017-07',
       '2017-10', '2018-01', '2018-04', '2018-07', '2018-10', '2019-01',
       '2019-04', '2019-07', '2019-10', '2020-01', '2020-04', '2020-07',
       '2020-10', '2021-01', '2021-04', '2021-07', '2021-10', '2022-01',
       '2022-04', '2022-07', '2022-10', '2023-01', '2023-04', '2023-07',
       '2023-10'], dtype=object)

In [13]:
df_test.quarter.unique()

array(['2024-01', '2024-04', '2024-07', '2024-10'], dtype=object)

In [19]:
!wget https://data.gharchive.org/2016-04-01-15.json.gz

--2025-02-12 21:09:35--  https://data.gharchive.org/2016-04-01-15.json.gz
Resolving data.gharchive.org (data.gharchive.org)... 172.67.168.206, 104.21.46.175, 2606:4700:3037::6815:2eaf, ...
Connecting to data.gharchive.org (data.gharchive.org)|172.67.168.206|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20877966 (20M) [application/gzip]
Saving to: ‘2016-04-01-15.json.gz’

2016-04-01-15.json. 100%[===================>]  19.91M  13.3MB/s    in 1.5s    

2025-02-12 21:09:37 (13.3 MB/s) - ‘2016-04-01-15.json.gz’ saved [20877966/20877966]



In [20]:
import gzip
import shutil

with gzip.open('2016-04-01-15.json.gz', 'rb') as f_in:
    with open('2016-04-01-15.json', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)


In [21]:
data = pd.read_json("2016-04-01-15.json", lines=True)

In [22]:
data.loc[2, 'repo']

{'id': 55240038,
 'name': 'michelpm1/base64encodewithnode',
 'url': 'https://api.github.com/repos/michelpm1/base64encodewithnode'}

In [5]:
data.tail()

,id,type,actor,repo,payload,public,created_at,org
11346,2489678838,IssuesEvent,"{'id': 1105372, 'login': 'ophian', 'gravatar_i...","{'id': 2627116, 'name': 's9y/Serendipity', 'ur...","{'action': 'closed', 'issue': {'url': 'https:/...",True,2015-01-01 15:59:59+00:00,"{'id': 1104713, 'login': 's9y', 'gravatar_id':..."
11347,2489678840,IssuesEvent,"{'id': 9343331, 'login': 'No-CQRT', 'gravatar_...","{'id': 25600089, 'name': 'No-CQRT/GooGuns', 'u...","{'action': 'opened', 'issue': {'url': 'https:/...",True,2015-01-01 15:59:59+00:00,NaN
11348,2489678842,DeleteEvent,"{'id': 2057932, 'login': 'LipkeGu', 'gravatar_...","{'id': 24847217, 'name': 'LipkeGu/OpenRA', 'ur...","{'ref': 'keep_source_clean', 'ref_type': 'bran...",True,2015-01-01 15:59:59+00:00,NaN
11349,2489678843,IssuesEvent,"{'id': 9343331, 'login': 'No-CQRT', 'gravatar_...","{'id': 25600089, 'name': 'No-CQRT/GooGuns', 'u...","{'action': 'opened', 'issue': {'url': 'https:/...",True,2015-01-01 15:59:59+00:00,NaN
11350,2489678844,IssuesEvent,"{'id': 9343331, 'login': 'No-CQRT', 'gravatar_...","{'id': 25600089, 'name': 'No-CQRT/GooGuns', 'u...","{'action': 'opened', 'issue': {'url': 'https:/...",True,2015-01-01 15:59:59+00:00,NaN


In [6]:
data.type.unique()

array(['CreateEvent', 'PushEvent', 'WatchEvent', 'ReleaseEvent',
       'PullRequestEvent', 'IssuesEvent', 'ForkEvent', 'GollumEvent',
       'IssueCommentEvent', 'DeleteEvent',
       'PullRequestReviewCommentEvent', 'CommitCommentEvent',
       'MemberEvent', 'PublicEvent'], dtype=object)